In [22]:
import numpy as np    
import pandas as pd  
from apyori import apriori 
ratings = pd.read_csv('C:\\Users\\User\\Desktop\\Workspace\\ML\\recommendation_engine\\input\\ml_small\\ratings.csv') 
movies = pd.read_csv('C:\\Users\\User\\Desktop\\Workspace\\ML\\recommendation_engine\\input\\ml_small\\movies.csv') 

movie_ratings = pd.merge(ratings[['userId','movieId','rating']],movies[['movieId','title']],on='movieId')
rating_threshold = 3
movie_ratings = movie_ratings.query('rating > @rating_threshold')

ratings_count = movie_ratings.groupby(by="title")['rating'].count().reset_index().rename(columns={'rating':'totalRatings'})[['title','totalRatings']]
all_ratings = pd.merge(movie_ratings,ratings_count,on='title',how='left')

votes_count_threshold = 20
ratings_top = all_ratings.query('totalRatings > @votes_count_threshold')
print('running for : ', ratings_top.shape)
#ratings_top.shape

if not ratings_top[ratings_top.duplicated(['userId','title'])].empty:
    ratings_top = ratings_top.drop_duplicates(['userId','title'])
    
#ratings_top.shape  - 66658,5
record_set = ratings_top.pivot(index='title',columns='userId',values='rating').fillna(0)
record_set =record_set.fillna(0)

def encode_units(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

ratings_top    
records_T = record_set.T.applymap(encode_units)   
records_T = records_T.fillna(0)
#records.head()

row,col = records_T.shape

#print(records_T['Alien (1979)'].sum(),records_T['Aliens (1986)'].sum())
#records_T['sum1'] =records_T['Alien (1979)']+ records_T['Aliens (1986)']+records_T['Matrix, The (1999)']
#records_T['sum2'] =records_T['Alien (1979)']+ records_T['Matrix, The (1999)']
#records_T = records_T.query('sum1 > 2')
#records_T.shape

running for :  (37557, 5)


In [23]:
records  = []
for i in range(0, row):  
    records.append([str(records_T.columns.values[j]) for j in range(0, col) if records_T.values[i,j] > 0])

#association_rules = apriori(records,min_support = 0.2,min_confidence=0.8,min_lift = 2,min_length=2) 
association_rules = apriori(records,min_support = 0.1,min_confidence=0.6,min_lift = 3,min_length=2) 
association_results = list(association_rules)

print(len(association_results))
for i in association_results:
    print(i)    

610
RelationRecord(items=frozenset({'Beauty and the Beast (1991)', 'Aladdin (1992)'}), support=0.10509031198686371, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Beauty and the Beast (1991)'}), items_add=frozenset({'Aladdin (1992)'}), confidence=0.7032967032967032, lift=3.3725015142337975)])
RelationRecord(items=frozenset({'Alien (1979)', 'Aliens (1986)'}), support=0.1297208538587849, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Alien (1979)'}), items_add=frozenset({'Aliens (1986)'}), confidence=0.6694915254237288, lift=4.160411622276029), OrderedStatistic(items_base=frozenset({'Aliens (1986)'}), items_add=frozenset({'Alien (1979)'}), confidence=0.8061224489795918, lift=4.160411622276029)])
RelationRecord(items=frozenset({'Alien (1979)', 'Terminator, The (1984)'}), support=0.10837438423645321, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Terminator, The (1984)'}), items_add=frozenset({'Alien (1979)'}), confidence=0.6055045871559633, lift=3.12

In [30]:

input_id = 1214
for item in association_results:
    pair = item[0]
    items = [x for x in pair]
    if  str(movies.loc[movies['movieId']==input_id]['title'].values[0]) in items[0] :
            #print('Rule->', items)
            print('Person who watched movie: ' + str(items[0]) + " also like" + str(items[1:]))
            print('Support: ' + str(item[1]))
            print('Confidence: ' + str(item[2][0][2]))
            print('Lift: ' + str(item[2][0][3]))            
            
        
    

Person who watched movie: Alien (1979) also like['Aliens (1986)']
Support: 0.1297208538587849
Confidence: 0.6694915254237288
Lift: 4.160411622276029
Person who watched movie: Alien (1979) also like['Terminator, The (1984)']
Support: 0.10837438423645321
Confidence: 0.6055045871559633
Lift: 3.1250194371015394
Person who watched movie: Alien (1979) also like['Matrix, The (1999)', 'Aliens (1986)']
Support: 0.1116584564860427
Confidence: 0.7727272727272728
Lift: 4.801948051948052
Person who watched movie: Alien (1979) also like['Star Wars: Episode V - The Empire Strikes Back (1980)', 'Star Wars: Episode IV - A New Hope (1977)']
Support: 0.12151067323481117
Confidence: 0.9024390243902439
Lift: 3.053252032520325
Person who watched movie: Alien (1979) also like['Star Wars: Episode V - The Empire Strikes Back (1980)', 'Star Wars: Episode VI - Return of the Jedi (1983)']
Support: 0.10673234811165845
Confidence: 0.9027777777777778
Lift: 3.054398148148148
Person who watched movie: Alien (1979) als

In [25]:
movies.loc[movies['title']=='Alien (1979)']

,movieId,title,genres
915,1214,Alien (1979),Horror|Sci-Fi
